# EDA
### Stellar Classification Dataset - SDSS17
https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17

#### Context
In astronomy, stellar classification is the classification of stars based on their spectral characteristics. The classification scheme of galaxies, quasars, and stars is one of the most fundamental in astronomy. The early cataloguing of stars and their distribution in the sky has led to the understanding that they make up our own galaxy and, following the distinction that Andromeda was a separate galaxy to our own, numerous galaxies began to be surveyed as more powerful telescopes were built. This datasat aims to classificate stars, galaxies, and quasars based on their spectral characteristics.

#### Content
The data consists of 100,000 observations of space taken by the SDSS (Sloan Digital Sky Survey). Every observation is described by 17 feature columns and 1 class column which identifies it to be either a star, galaxy or quasar.

obj_ID = Object Identifier, the unique value that identifies the object in the image catalog used by the CAS\
alpha = Right Ascension angle (at J2000 epoch)\
delta = Declination angle (at J2000 epoch)\
u = Ultraviolet filter in the photometric system\
g = Green filter in the photometric system\
r = Red filter in the photometric system\
i = Near Infrared filter in the photometric system\
z = Infrared filter in the photometric system\
run_ID = Run Number used to identify the specific scan\
rereun_ID = Rerun Number to specify how the image was processed\
cam_col = Camera column to identify the scanline within the run\
field_ID = Field number to identify each field\
spec_obj_ID = Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)\
class = object class (galaxy, star or quasar object)\
redshift = redshift value based on the increase in wavelength\
plate = plate ID, identifies each plate in SDSS\
MJD = Modified Julian Date, used to indicate when a given piece of SDSS data was taken\
fiber_ID = fiber ID that identifies the fiber that pointed the light at the focal plane in each observation\

#### Citation
fedesoriano. (January 2022). Stellar Classification Dataset - SDSS17. Retrieved [Date Retrieved] from https://www.kaggle.com/fedesoriano/stellar-classification-dataset-sdss17.

Acknowledgements
The data released by the SDSS is under public domain. Its taken from the current data release RD17.

More information about the license: http://www.sdss.org/science/image-gallery/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sklearn 
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
np.random.seed = 42

In [ ]:
## W tej komórce są operacje dokonywane na ramce danych w pozostałej części tego pliku
# df_original = pd.read_csv('star_classification.csv')
# df = df_original.copy()

# df = df.loc[:, df.columns != 'rerun_ID']
# df = df[df['u'] > 0]

# df_correlation = df.loc[:, df.columns != 'class'].corr()
# df_correlation_GALAXY = df[df['class']=='GALAXY'].loc[:, df.columns != 'class'].corr()
# df_correlation_QSO = df[df['class']=='QSO'].loc[:, df.columns != 'class'].corr()
# df_correlation_STAR = df[df['class']=='STAR'].loc[:, df.columns != 'class'].corr()

### Ten kod poniżej to zamiana zmiennej kategorycznej numeryczną, nie był używany
###one-hot for class
## df['is'] = df['class']
## df = pd.get_dummies(df, columns = ['is'], dtype=float)
## df.head()

## Podstawowe informacje o ramce danych

In [ ]:
#df_original = pd.read_csv('C:\\Users\\HP\\Downloads\\archive\\star_classification.csv')
df_original = pd.read_csv(".//test_sample_team4.csv")
df_original.head()

In [ ]:
df_original.info()

nie ma braków danych w żadnej kolumnie

In [ ]:
df_original.describe()

In [ ]:
df_original.hist(bins = 40, figsize=(18, 12))
plt.show()

In [ ]:
df_original['class'].value_counts().plot.bar().set(title='Rozkład zmiennej "class"')
plt.show()
df_original['class'].value_counts()

#### zmienna class posiada trzy unikalne wartości: star, galaxy i quasar, z czego obiektów typu galaxy jest trzy razy więcej niż tyou star i trzy razy więcej niż typu quasar

## Wstępna korekta ramki danych

In [ ]:
df = df_original.copy()

In [ ]:
df['rerun_ID'].value_counts()

Wszystkie obserwacje w kolumnie 'rerun_ID' są takie same, więc tę cechę można pominąć, bo nie niesie ze sobą żadnych informacji.

In [ ]:
df = df.loc[:, df.columns != 'rerun_ID']
df.head()

In [ ]:
df.duplicated().value_counts()

W ramce danych nie występują zduplikowane obserwacje.

przyjrzyjmy się zmiennym u, g, z. z wykresów oraz z opisu powyżej można wywnioskować, że istnieje wiersz z bardzo odstającymi wartościami

In [ ]:
## boxploty dla zmiennych u, g, z, rerun_ID
fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

axs[0].boxplot(df["u"])
axs[0].set_title('u')

axs[1].boxplot(df['g'])
axs[1].set_title('g')

axs[2].boxplot(df['z'])
axs[2].set_title('z')

In [ ]:
### jeden outlier, psuje wszystko -> do usunięcia!!!
df = df[df['u'] > 0]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

axs[0].boxplot(df["u"])
axs[0].set_title('u')

axs[1].boxplot(df['g'])
axs[1].set_title('g')

axs[2].boxplot(df['z'])
axs[2].set_title('z')

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

axs[0].hist(df["u"], bins = 40)
axs[0].set_title('u')

axs[1].hist(df['g'], bins = 40)
axs[1].set_title('g')

axs[2].hist(df['z'], bins = 40)
axs[2].set_title('z')
plt.show()

po usunięciu outliera dopiero widać, że powyższe zmienne mają rozkład normalny

## Analiza danych

In [ ]:
df_correlation = df.loc[:, df.columns != 'class'].corr()
df_correlation_GALAXY = df[df['class']=='GALAXY'].loc[:, df.columns != 'class'].corr()
df_correlation_QSO = df[df['class']=='QSO'].loc[:, df.columns != 'class'].corr()
df_correlation_STAR = df[df['class']=='STAR'].loc[:, df.columns != 'class'].corr()
fig = plt.figure(figsize=(16,16), dpi = 256)
sns.heatmap(df_correlation, annot = True, fmt = '.2f')
fig = plt.figure(figsize=(16,16), dpi = 256)
sns.heatmap(df_correlation_GALAXY, annot = True, fmt = '.2f')
plt.show()
fig = plt.figure(figsize=(16,16), dpi = 256)
sns.heatmap(df_correlation_QSO, annot = True, fmt = '.2f')
plt.show()
fig = plt.figure(figsize=(16,16), dpi = 256)
sns.heatmap(df_correlation_STAR, annot = True, fmt = '.2f')
plt.show()

In [21]:
sns.pairplot(df.iloc[0:1000,:], hue='class')
# sns.pairplot(df, hue='class')
plt.tight_layout() # poszczególnym wykresom przyjrzemy się w kolejnych częściach

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.inspection import permutation_importance

# tmp_df = df.drop('class', axis=1).iloc[0:100,:]
# X_train, y_train = train_test_split(tmp_df, test_size=0.5, random_state=42)
# rf = RandomForestRegressor(n_estimators=100)
# rf.fit(X_train, y_train)
# sorted_idx = rf.feature_importances_.argsort()
# plt.barh(tmp_df.columns[sorted_idx], rf.feature_importances_[sorted_idx])

In [ ]:
# perm_importance = permutation_importance(rf, X_train, y_train)
# sorted_idx = perm_importance.importances_mean.argsort()
# plt.barh(tmp_df.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])

# EDA cz. 1

In [ ]:
def histograms(colname):
    class_star = df[df['class'] == 'STAR']
    class_galaxy = df[df['class'] == 'GALAXY']
    class_quasar = df[df['class'] == 'QSO']

    fig, axs = plt.subplots(1, 4, figsize=(15, 5), sharey=True)

    axs[0].hist(class_star[colname], bins=30, color='blue', alpha=0.7)
    axs[0].set_title('Star')

    axs[1].hist(class_galaxy[colname], bins=30, color='green', alpha=0.7)
    axs[1].set_title('Galaxy')

    axs[2].hist(class_quasar[colname], bins=30, color='red', alpha=0.7)
    axs[2].set_title('Quasar')

    axs[3].hist(df[colname], bins=30, color='orange', alpha=0.7)
    axs[3].set_title('df')

    plt.show()

## obj_ID, run_ID

In [ ]:
plt.plot(df['obj_ID'], df['run_ID'])

jak widać powyżej oraz z macierzy korelacji, istnieje bardzo silna zależność między zmiennymi obj_Id oraz run_Id
-> jedna z kolumn musi zostać usunięta

In [ ]:
histograms('obj_ID')

In [ ]:
histograms('run_ID')

równie silna korelacja występuje dla kolumn spec_obj_ID oraz plate

In [ ]:
plt.plot(df['spec_obj_ID'], df['plate'])  ### jedna z tych zmiennych też do usunięcia

## alpha i delta
alpha = Right Ascension angle (at J2000 epoch)
delta = Declination angle (at J2000 epoch)

spójrzmy na zmienne alpha i delta. Widzimy, że zmienna alpha przyjmuje bardzo mało wartości w przedziałach 60 - 100 oraz 250 - 300. Większość wartości w tych przedziałach jest typu star.

In [ ]:
###alpha
histograms('alpha')

In [ ]:
k = df[((df['alpha'] > 60) & (df['alpha'] < 100)) | ((df['alpha'] > 260) & (df['alpha'] < 300))]
star = k[k['class'] == 'STAR']
galaxy = k[k['class'] == 'GALAXY']
quasar = k[k['class'] == 'QSO']

rows = [len(star), len(galaxy), len(quasar)]
names = ['star', 'galaxy', 'quasar']

plt.bar(names, rows)
plt.show()

In [ ]:
###delta
histograms('delta')

zmienna delta przypomina rozkład normalny dla każdego z powyższsych wykresów. Wyjątkiem jest widoczny 'pik' w okolicy wartości 0.

In [ ]:
sns.scatterplot(x = df.alpha, y = df.delta, data = df)

In [ ]:
sns.scatterplot(x = df.alpha, y = df.delta, data = df, hue = 'class')
#zależnośc między alfa i delta

zmienne alpha i delta mają bardzo mały współczynnik korelacji z innymi kolumnami. Największy współczynnik mają w korelacji samymi z sobą, gdy nie uwzględniamy podziału na klasy (współczynnik wynosi 0.14). Nieco większy wynik (0.16) otrzymujemy, gdy rozpatrujemy zmienny typu 'STAR'.

## filters in the photometric system
### u - ultraviolet
### g - green
### i - near infrared
### z - infrared
### r - red

In [ ]:
g = sns.pairplot(data=df[['u', 'g', 'i', 'z', 'r', 'class']],
                 hue='class')
plt.show()

Powyższe zmienne są bardzo silnie skorelowane, co stanowi podstawę do usunięcia części z nich
Przyjrzyjmy się poniższym histogramom - dla każdej zmiennej rozkład w poszczególnej klasie jest bardzo podobny. Histogramy różnią się jednak w kategoriach. Z wykresów poniżej wynika, że każda nowa zmienna nie wprowadza żadnych nowych informacji.

In [ ]:
histograms('u')

In [ ]:
histograms('g')

In [ ]:
histograms('i')

In [ ]:
histograms('z')

In [ ]:
histograms('r')

In [ ]:
point_limit=100000 # ramka danych ma 100000 obserwacji, kolekcje wykresów punktowych się długo rysują przy tej liczbie

def standard_distribution_info(variable_name):
    fig, ax = plt.subplots(4, 1, figsize=(10, 10))
    sns.histplot(data=df, x=variable_name, color='#17becf', ax=ax[0]).set(title='All classes')
    sns.histplot(data=df[df['class']=='GALAXY'], x=variable_name, color='#1f77b4', ax=ax[1]).set(title='GALAXY')
    sns.histplot(data=df[df['class']=='QSO'], x=variable_name, color='#ff7f0e', ax=ax[2]).set(title='QSO')
    sns.histplot(data=df[df['class']=='STAR'], x=variable_name, color='#2ca02c', ax=ax[3]).set(title='STAR')
    fig.tight_layout(pad=5.0)
    fig.show()

def standard_boxplot_info(variable_name):
    fig, ax = plt.subplots(4, 1, figsize=(10, 10))
    sns.boxplot(data=df, x=variable_name, color='#17becf', ax=ax[0]).set(title='All classes')
    sns.boxplot(data=df[df['class']=='GALAXY'], x=variable_name, color='#1f77b4', ax=ax[1]).set(title='GALAXY')
    sns.boxplot(data=df[df['class']=='QSO'], x=variable_name, color='#ff7f0e', ax=ax[2]).set(title='QSO')
    sns.boxplot(data=df[df['class']=='STAR'], x=variable_name, color='#2ca02c', ax=ax[3]).set(title='STAR')
    fig.tight_layout(pad=5.0)
    fig.show()
    boxplot_info(variable_name)
    
def boxplot_info(variable_name):
    # outliery są znajdywane za pomocą metody 1.5IQR
    print('All classes:')
    selected_df = df[variable_name]
    print('mediana = %s' %np.median(selected_df))
    print('średnia = %s' %np.mean(selected_df))
    Q1 = np.percentile(selected_df, 25)
    Q3 = np.percentile(selected_df, 75)
    IQR = Q3-Q1
    print('Q1 = %s' %Q1,'Q3 = %s' %Q3)
    print('outlier_lower_bound = %s' %(Q1-(1.5*IQR)))
    print('outlier_upper_bound = %s' %(Q3+(1.5*IQR)))
    print('outlier_count = %s' %selected_df[(selected_df < Q1-(1.5*IQR)) | (selected_df > Q3+(1.5*IQR))].count())
    print('GALAXY:')
    selected_df = df[df['class']=='GALAXY'][variable_name]
    print('mediana = %s' %np.median(selected_df))
    print('średnia = %s' %np.mean(selected_df))
    Q1 = np.percentile(selected_df, 25)
    Q3 = np.percentile(selected_df, 75)
    IQR = Q3-Q1
    print('Q1 = %s' %Q1,'Q3 = %s' %Q3)
    print('outlier_lower_bound = %s' %(Q1-(1.5*IQR)))
    print('outlier_upper_bound = %s' %(Q3+(1.5*IQR)))
    print('outlier_count = %s' %selected_df[(selected_df < Q1-(1.5*IQR)) | (selected_df > Q3+(1.5*IQR))].count())
    print('QSO:')
    selected_df = df[df['class']=='QSO'][variable_name]
    print('mediana = %s' %np.median(selected_df))
    print('średnia = %s' %np.mean(selected_df))
    Q1 = np.percentile(selected_df, 25)
    Q3 = np.percentile(selected_df, 75)
    IQR = Q3-Q1
    print('Q1 = %s' %Q1,'Q3 = %s' %Q3)
    print('outlier_lower_bound = %s' %(Q1-(1.5*IQR)))
    print('outlier_upper_bound = %s' %(Q3+(1.5*IQR)))
    print('outlier_count = %s' %selected_df[(selected_df < Q1-(1.5*IQR)) | (selected_df > Q3+(1.5*IQR))].count())
    print('STAR:')
    selected_df = df[df['class']=='STAR'][variable_name]
    print('mediana = %s' %np.median(selected_df))
    print('średnia = %s' %np.mean(selected_df))
    Q1 = np.percentile(selected_df, 25)
    Q3 = np.percentile(selected_df, 75)
    IQR = Q3-Q1
    print('Q1 = %s' %Q1,'Q3 = %s' %Q3)
    print('outlier_lower_bound = %s' %(Q1-(1.5*IQR)))
    print('outlier_upper_bound = %s' %(Q3+(1.5*IQR)))
    print('outlier_count = %s' %selected_df[(selected_df < Q1-(1.5*IQR)) | (selected_df > Q3+(1.5*IQR))].count())
    
def standard_correlation_visualisation(variable_name, limit):
    sns.pairplot(df.head(limit), y_vars=variable_name, x_vars=df.columns.values[:5], hue='class')
    sns.pairplot(df.head(limit), y_vars=variable_name, x_vars=df.columns.values[5:10], hue='class')
    sns.pairplot(df.head(limit), y_vars=variable_name, x_vars=df.columns.values[10:15], hue='class')
    sns.pairplot(df.head(limit), y_vars=variable_name, x_vars=df.columns.values[15:18], hue='class')
    plt.show()
    fig, ax = plt.subplots(2, 2, figsize=(10, 10))
    df_correlation[variable_name].plot.bar(color='#17becf', ylim=(-1, 1), ax=ax[0, 0]).set(title='All classes correlations') 
    df_correlation_GALAXY[variable_name].plot.bar(color='#1f77b4', ylim=(-1, 1), ax=ax[0, 1]).set(title='GALAXY correlations')
    df_correlation_QSO[variable_name].plot.bar(color='#ff7f0e', ylim=(-1, 1), ax=ax[1, 0]).set(title='QSO correlations')
    df_correlation_STAR[variable_name].plot.bar(color='#2ca02c', ylim=(-1, 1), ax=ax[1, 1]).set(title='STAR correlations')
    ax[0, 0].axhline(y = 0, color = 'black', linewidth = 0.5, linestyle = '-')
    ax[0, 1].axhline(y = 0, color = 'black', linewidth = 0.5, linestyle = '-') 
    ax[1, 0].axhline(y = 0, color = 'black', linewidth = 0.5, linestyle = '-')
    ax[1, 1].axhline(y = 0, color = 'black', linewidth = 0.5, linestyle = '-') 
    fig.tight_layout(pad=5.0)
    fig.show()
    print('korelacja:')
    print('all_classes: \n%s' %df_correlation.sort_values(by=[variable_name], ascending=False)[variable_name])
    print('GALAXY: \n%s' %df_correlation_GALAXY.sort_values(by=[variable_name], ascending=False)[variable_name])
    print('QSO: \n%s' %df_correlation_QSO.sort_values(by=[variable_name], ascending=False)[variable_name])
    print('STAR: \n%s' %df_correlation_STAR.sort_values(by=[variable_name], ascending=False)[variable_name])

# cam_col
 Camera column to identify the scanline within the run
 
 Jest to zmienna o wartościach dyskretnych.
 Jej rozkład jest podobny dla każdej z klas; wartości brzegowe są rzadsze.
 Nie widać wyraźnego związku 'cam_col' z żadną z pozostałych zmiennych.

In [ ]:
standard_distribution_info('cam_col')
df['cam_col'].value_counts()

In [ ]:
standard_boxplot_info('cam_col')

In [ ]:
standard_correlation_visualisation('cam_col', point_limit)

# field_ID
Field number to identify each field

Zmienne najczęściej przyjmują wartości w okolicach 100, a od 200 wraz ze wzrostem wartości maleje szansa na jej osiągnięcie.
To powoduje istnienie dużej liczby, bo około 5%, wartości odstających. Ta tendencja utrzymuje się na przestrzeni wszystkich klas.
Ta kolumna jest skorelowana na poziomie około -0.16 z 'alpha' i 'delta'

In [ ]:
standard_distribution_info('field_ID')
df['field_ID'].value_counts()

In [ ]:
standard_boxplot_info('field_ID')

In [ ]:
standard_correlation_visualisation('field_ID', point_limit)

# spec_obj_ID
Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)

Wartości są rozłożone na przedziale 0 - ~14e19, przy czym GALAXY ma bardziej wypośrodkowany rozkład, QSO jest prawoskośne, a STAR jest lewoskośne.  
Jedynie klasa STAR ma wartości odstające, ale jest ich tylko 18.  
'plate' i 'MJD' są skorelowane z tą zmienną blisko 1, a więc te kolumny nadają się do odrzucenia.  
'r', 'z', 'i' są też silnie skorelowane (0.4-0.6).  
'g', 'redshift', 'u' dodatkowo korelacje > 0.55 dla klasy GALAXY.

In [ ]:
standard_distribution_info('spec_obj_ID')
df['spec_obj_ID'].value_counts()

In [ ]:
standard_boxplot_info('spec_obj_ID')

In [ ]:
standard_correlation_visualisation('spec_obj_ID', point_limit)

# class
object class (galaxy, star or quasar object)

Najliczniejsza jest klasa GALAXY, bo jej licznośc jest około trzykrotnie większa od pozostałych klas.  
Ze względu na niewielki startowy rozmiar ramki danych (tylko 18 kolumn) dodanie kilku nowych kolumn i zwiększenie wymiarowości nie powinno znacząco pogorszyć modelu, a zatem zamiana tej zmiennej kategorycznej na numeryczną metodą one-hot wydaje się trafnym pomysłem.

In [ ]:
sns.histplot(data=df, x='class', color='#17becf').set(title='All classes')
df['class'].value_counts()

# redshift
redshift value based on the increase in wavelength

QSO jako rodzaj galaktyki ma podobny rozkład co GALAXY, tzn. poczerwienienie jest dodatnie. Warto zauważyć, że GALAXY przyjmuje wartości w większości w przedziale 0-1, a QSO 0-3. STAR natomiast ma odmienny rozkład; Gaussa blisko skupiony wokół zera. To powoduje, że łączny rozkład zmiennej 'redshift' najczęściej (i to znacząco) przyjmuje wartości bliskie zeru.   
Ilość wartości odstających jest duża, z czego ze względu na różny zakres zmiennej w podziale na klasy wartości odstające dla poszczególnych klas różnią się od wartości odstających dla rozkładu łącznego (głównie przez wpływ rozkładu QSO).  
'u', 'g', 'r', 'i', 'z' są skorelowane w klasach GALAXY i QSO (w GALAXY znacząco, bo >0.6, w QSO znacznie mniej, bo ~0.17-~0.32), a GALAXY jest dodatkowo silnie skorelowane z 'MJD', 'spec_obj_ID', 'plate' (>0.6).  
Ze względu na bardzo charakterystyczne rozkłady zmiennej w podziale na klasy można wysnuć hipotezę o niebagatelnym znaczeniu cechy 'redshift' w zadaniu klasyfikacji obiektu w oparciu o analizę przyjmowanych wartości przez tę zmienną.

In [ ]:
standard_distribution_info('redshift')
df['redshift'].value_counts()

In [ ]:
standard_boxplot_info('redshift')

In [ ]:
standard_correlation_visualisation('redshift', point_limit)

# plate 
plate ID, identifies each plate in SDSS

Ta zmienna ma podobny rozkład co 'space_obj_ID'. Co więcej, pozostałe parametry są również podobne do wspomnianej zmiennej. Powodem takiego zjawiska jest zaobserwowany wcześniej fakt, mianowicie; 'space_obj_ID' jak i 'plate' są skorelowane ze sobą ze współczynnikiem korelacji równym 1. Zdublowanie informacji poprzez podanie identycznych cech do modelu może skutkować brakiem równowagi w porónaniu do pozostałych cech w zadaniu klasyfikacji, a zatem jedna z tych kolumn powinna zostać usunięta z ramki danych.

In [ ]:
standard_distribution_info('plate')
df['plate'].value_counts()

In [ ]:
standard_boxplot_info('plate')

In [ ]:
standard_correlation_visualisation('plate', point_limit)

# MJD
Modified Julian Date, used to indicate when a given piece of SDSS data was taken

Podobnie jak we wcześniej opisanej kolumnie (patrz: 'plate') skorelowanie ze zmienną 'space_obj_ID' i 'plate' jest bardzo wysokie (>0.96). Wynika z tego, że pozostałe parametry tej zmiennej są prawie identyczne (co można sprawdzić w sposób empiryczny). Wniosek z tego jest następujący: 2 z 3 silnie skorelowanych kolumn ('space_obj_ID', 'plate', 'MJD') na pewno powinny zostać usunięte z ramki danych, bo ich obecność nie niesie ze sobą istnienia nowych informacji.

In [ ]:
standard_distribution_info('MJD')
df['MJD'].value_counts()

In [ ]:
standard_boxplot_info('MJD')

In [ ]:
standard_correlation_visualisation('MJD', point_limit)

# fiber_ID
fiber ID that identifies the fiber that pointed the light at the focal plane in each observati

Rozkład zmiennej 'fiber_ID' jest podobny do jednostajnego z tym wyjątkiem, że dla klasy GALAXY i STAR rozkład wartości większych niż 600 jest dalej jednostajny, jednak ilość ich wystąpień jest mniejsza.  
W ramce danych nie ma obecnych wartości odstających jeśli chodzi o tę zmienną.  
Wartości 'fiber_ID' są w przybliżeniu <600 dla 'MJD' <5500, 'plate' <4000 i 'space_obj_ID' <0,5. Tę samą zależność można zaobserwować w zmiennych 'alpha' dla wartości 100+-10 i 275+-10 oraz 'delta' <-5 i >75 (będących głównie obserwacjami klasy STAR). Ta zależność przenosi się też poniekąd na pozostałe odpowiednio silnie skorelowane zmienne.  
Niewielka korelacja może być zauważona dla 'u', 'g', 'r', 'i', 'z', 'space_obj_ID', 'plate', 'MJD', 'redshift' dla klasy GALAXY (0.2-0.26) i 'space_obj_ID', 'plate', 'MJD' dla klasy STAR (0.28).

In [ ]:
standard_distribution_info('fiber_ID')
df['fiber_ID'].value_counts()

In [ ]:
standard_boxplot_info('fiber_ID')

In [ ]:
standard_correlation_visualisation('fiber_ID', point_limit)

### Dodatkowo
Zmienne mają różny zakres i różne rozkłady, dlatego będzie trzeba je znormalizować/zestandardyzować.